# Create Project CERES Deliverables for Tennessee farm and home science (agrtfn)

In [74]:
# importing and options
import time
from pathlib import Path

import pandas as pd
from ipywidgets import IntProgress, Label, VBox
from IPython.display import display

# == display 95% width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# set pandas option to display wider column info, use -1 to expand to largest necessary width
pd.set_option('max_colwidth', -1)

## Load data from CSV

In [75]:
# exported Google Docs spreadsheet as csv
meta_df = pd.read_csv('data/agrtfn_crl_deliverables.csv')
meta_df.head(5)

,adminDB,title,date_from_utk_metadata_app,crl_deliverable_directory
0,0012_004266_000001,"Tennessee farm news, January 2, 1925","January 2, 1921",NaN
1,0012_004266_000002,"Tennessee farm news, January 9, 1921","January 9, 1921",NaN
2,0012_004266_000003,"Tennessee farm news, July 4, 1921","July 13, 1925",NaN
3,0012_004266_000004,"Tennessee farm news, July 11, 1921","July 13, 1925",NaN
4,0012_004266_000005,"Tennessee farm news, July 16, 1921","July 13, 1925",NaN


In [76]:
# drop unfilled crl_deliverable_directory
meta_df = meta_df.drop(labels='crl_deliverable_directory', axis=1)
meta_df.head(5)

,adminDB,title,date_from_utk_metadata_app
0,0012_004266_000001,"Tennessee farm news, January 2, 1925","January 2, 1921"
1,0012_004266_000002,"Tennessee farm news, January 9, 1921","January 9, 1921"
2,0012_004266_000003,"Tennessee farm news, July 4, 1921","July 13, 1925"
3,0012_004266_000004,"Tennessee farm news, July 11, 1921","July 13, 1925"
4,0012_004266_000005,"Tennessee farm news, July 16, 1921","July 13, 1925"


## Split date from title

In [77]:
# split by comma and space then get everything after first match
meta_df['title'].str.split(', ', n=1).str[1].head(5)

0    January 2, 1925
1    January 9, 1921
2    July 4, 1921   
3    July 11, 1921  
4    July 16, 1921  
Name: title, dtype: object

In [78]:
# add date_range to Dataframe by rsplitting on comma and taking the last field
meta_df['date'] = meta_df['title'].str.split(', ', n=1).str[1]
meta_df.head(5)

,adminDB,title,date_from_utk_metadata_app,date
0,0012_004266_000001,"Tennessee farm news, January 2, 1925","January 2, 1921","January 2, 1925"
1,0012_004266_000002,"Tennessee farm news, January 9, 1921","January 9, 1921","January 9, 1921"
2,0012_004266_000003,"Tennessee farm news, July 4, 1921","July 13, 1925","July 4, 1921"
3,0012_004266_000004,"Tennessee farm news, July 11, 1921","July 13, 1925","July 11, 1921"
4,0012_004266_000005,"Tennessee farm news, July 16, 1921","July 13, 1925","July 16, 1921"


## Convert date into yyyy, mm, dd

In [79]:
# split date range by spaces and get the first month
meta_df['date'].str.split().str[0].head(5)

0    January
1    January
2    July   
3    July   
4    July   
Name: date, dtype: object

In [80]:
# add the month to the DataFrame
meta_df['Month'] = meta_df['date'].str.split().str[0]
meta_df.head(5)

,adminDB,title,date_from_utk_metadata_app,date,Month
0,0012_004266_000001,"Tennessee farm news, January 2, 1925","January 2, 1921","January 2, 1925",January
1,0012_004266_000002,"Tennessee farm news, January 9, 1921","January 9, 1921","January 9, 1921",January
2,0012_004266_000003,"Tennessee farm news, July 4, 1921","July 13, 1925","July 4, 1921",July
3,0012_004266_000004,"Tennessee farm news, July 11, 1921","July 13, 1925","July 11, 1921",July
4,0012_004266_000005,"Tennessee farm news, July 16, 1921","July 13, 1925","July 16, 1921",July


In [81]:
# split yyyy from date range and add to Dataframe
meta_df['yyyy'] = meta_df['date'].str.split().str[-1]
meta_df.head(5)

,adminDB,title,date_from_utk_metadata_app,date,Month,yyyy
0,0012_004266_000001,"Tennessee farm news, January 2, 1925","January 2, 1921","January 2, 1925",January,1925
1,0012_004266_000002,"Tennessee farm news, January 9, 1921","January 9, 1921","January 9, 1921",January,1921
2,0012_004266_000003,"Tennessee farm news, July 4, 1921","July 13, 1925","July 4, 1921",July,1921
3,0012_004266_000004,"Tennessee farm news, July 11, 1921","July 13, 1925","July 11, 1921",July,1921
4,0012_004266_000005,"Tennessee farm news, July 16, 1921","July 13, 1925","July 16, 1921",July,1921


In [82]:
# convert month to mm, e.g. January -> 01
str(time.strptime(meta_df['Month'][0], '%B').tm_mon).zfill(2)

'01'

In [83]:
# create a list of what the mm column should be
# due to formatting issues getting time.strptime to work on a Pandas column
mm_list = []
for index, value in enumerate(meta_df['Month']):
    try:
        mm = str(time.strptime(value, '%B').tm_mon).zfill(2)
    except ValueError:
        print(index, value)
    mm_list.append(mm)
mm_list[:6]

['01', '01', '07', '07', '07', '07']

In [84]:
# add mm column to DataFrame using list
meta_df['mm'] = mm_list
meta_df.tail(5)

,adminDB,title,date_from_utk_metadata_app,date,Month,yyyy,mm
3446,0012_004266_003448,"Tennessee farm and home news, September 15, 1947",NaN,"September 15, 1947",September,1947,09
3447,0012_004266_003449,"Tennessee farm and home news, April 2, 1951",NaN,"April 2, 1951",April,1951,04
3448,0012_004266_003450,"Tennessee farm and home news, January 27, 1947",NaN,"January 27, 1947",January,1947,01
3449,0012_004266_003452,"Tennessee farm and home news, July 1, 1935",NaN,"July 1, 1935",July,1935,07
3450,0012_004266_003453,"Tennessee farm and home news, March 20, 1939",NaN,"March 20, 1939",March,1939,03


In [85]:
# add dd to DataFrame
meta_df['dd'] = meta_df['date'].str.split().str[-2].str.strip(',').str.zfill(2)
meta_df.head(5)

,adminDB,title,date_from_utk_metadata_app,date,Month,yyyy,mm,dd
0,0012_004266_000001,"Tennessee farm news, January 2, 1925","January 2, 1921","January 2, 1925",January,1925,01,02
1,0012_004266_000002,"Tennessee farm news, January 9, 1921","January 9, 1921","January 9, 1921",January,1921,01,09
2,0012_004266_000003,"Tennessee farm news, July 4, 1921","July 13, 1925","July 4, 1921",July,1921,07,04
3,0012_004266_000004,"Tennessee farm news, July 11, 1921","July 13, 1925","July 11, 1921",July,1921,07,11
4,0012_004266_000005,"Tennessee farm news, July 16, 1921","July 13, 1925","July 16, 1921",July,1921,07,16


## Map old to new directory

In [86]:
# explicitly define adminDB as old_directory_name
meta_df['old_directory_name'] = meta_df['adminDB']
meta_df.head(5)

,adminDB,title,date_from_utk_metadata_app,date,Month,yyyy,mm,dd,old_directory_name
0,0012_004266_000001,"Tennessee farm news, January 2, 1925","January 2, 1921","January 2, 1925",January,1925,01,02,0012_004266_000001
1,0012_004266_000002,"Tennessee farm news, January 9, 1921","January 9, 1921","January 9, 1921",January,1921,01,09,0012_004266_000002
2,0012_004266_000003,"Tennessee farm news, July 4, 1921","July 13, 1925","July 4, 1921",July,1921,07,04,0012_004266_000003
3,0012_004266_000004,"Tennessee farm news, July 11, 1921","July 13, 1925","July 11, 1921",July,1921,07,11,0012_004266_000004
4,0012_004266_000005,"Tennessee farm news, July 16, 1921","July 13, 1925","July 16, 1921",July,1921,07,16,0012_004266_000005


In [87]:
# add new_directory_name from yyyy, mm, and dd columns
meta_df['new_directory_name'] = meta_df['yyyy'].map(str) + '_' + meta_df['mm'].map(str) + '_' + meta_df['dd'].map(str)
meta_df.head(5)

,adminDB,title,date_from_utk_metadata_app,date,Month,yyyy,mm,dd,old_directory_name,new_directory_name
0,0012_004266_000001,"Tennessee farm news, January 2, 1925","January 2, 1921","January 2, 1925",January,1925,01,02,0012_004266_000001,1925_01_02
1,0012_004266_000002,"Tennessee farm news, January 9, 1921","January 9, 1921","January 9, 1921",January,1921,01,09,0012_004266_000002,1921_01_09
2,0012_004266_000003,"Tennessee farm news, July 4, 1921","July 13, 1925","July 4, 1921",July,1921,07,04,0012_004266_000003,1921_07_04
3,0012_004266_000004,"Tennessee farm news, July 11, 1921","July 13, 1925","July 11, 1921",July,1921,07,11,0012_004266_000004,1921_07_11
4,0012_004266_000005,"Tennessee farm news, July 16, 1921","July 13, 1925","July 16, 1921",July,1921,07,16,0012_004266_000005,1921_07_16


In [88]:
# verify that we have a matching number of old_ and new_directory_name(s)
meta_df.count()

adminDB                       3451
title                         3451
date_from_utk_metadata_app    1517
date                          3451
Month                         3451
yyyy                          3451
mm                            3451
dd                            3451
old_directory_name            3451
new_directory_name            3451
dtype: int64

In [89]:
dir_names = []
extra_dir_names = []
for new_directory_name in meta_df.new_directory_name:
    if new_directory_name not in dir_names:
        dir_names.append(new_directory_name)
    else:
        print(new_directory_name)
        extra_dir_names.append(new_directory_name)

1925_08_17
1930_10_27
1933_04_03
1937_05_03
1947_03_10
1935_07_01


In [90]:
for index, value in enumerate(meta_df.new_directory_name):
    if value in extra_dir_names:
        print('*****')
        print(value)
        print(meta_df.iloc[index])
        print('=====')

*****
1925_08_17
adminDB                       0012_004266_000214                  
title                         Tennessee farm news, August 17, 1925
date_from_utk_metadata_app    August 17, 1925                     
date                          August 17, 1925                     
Month                         August                              
yyyy                          1925                                
mm                            08                                  
dd                            17                                  
old_directory_name            0012_004266_000214                  
new_directory_name            1925_08_17                          
Name: 213, dtype: object
=====
*****
1925_08_17
adminDB                       0012_004266_000215                  
title                         Tennessee farm news, August 17, 1925
date_from_utk_metadata_app    August 17, 1925                     
date                          August 17, 1925                   

## Convert pages value to integer

In [14]:
# Add the number of pages, without " pages", from the extent field
df_tn_farm_and_home_science['pages'] = df_tn_farm_and_home_science['extent'].str.split().str[0]
df_tn_farm_and_home_science.head(5)

,adminDB,title,extent,date_range,Month,yyyy,mm,old_directory_name,new_directory_name,pages
0,0012_004267_000001,"Tennessee farm and home science, progress report 1, January - March 1952",12 pages,January - March 1952,January,1952,01,0012_004267_000001,1952_01,12
1,0012_004267_000002,"Tennessee farm and home science, progress report 2, April - June 1952",12 pages,April - June 1952,April,1952,04,0012_004267_000002,1952_04,12
2,0012_004267_000003,"Tennessee farm and home science, progress report 3, July - September 1952",12 pages,July - September 1952,July,1952,07,0012_004267_000003,1952_07,12
3,0012_004267_000004,"Tennessee farm and home science, progress report 4, October - December 1952",12 pages,October - December 1952,October,1952,10,0012_004267_000004,1952_10,12
4,0012_004267_000005,"Tennessee farm and home science, progress report 5, January - March 1953",12 pages,January - March 1953,January,1953,01,0012_004267_000005,1953_01,12


In [15]:
df_tn_farm_and_home_science['new_directory_name'][0]

'1952_01'

## Batch process directories

In [16]:
# create class for processing individual volumes
class Agrtfn:
    def __init__(self, directory_path):
        self.directory_path = Path(directory_path)
        self.number_of_pages = int(df_tn_farm_and_home_science[df_tn_farm_and_home_science["old_directory_name"].str.contains(directory_path.name)]['pages'].to_string(index=False))
        
    def get_image_paths_list(self):
        
        image_paths_list = sorted(self.directory_path.glob('*.tif'))
        
        # delete all macOS index files that start with '.'
        deleted_dot_files = False
        for image_path in image_paths_list:
            if image_path.name.startswith('.'):
                image_path.unlink()  # delete
                deleted_dot_files = True
        if deleted_dot_files:  # re-create the list
            image_paths_list = sorted(self.directory_path.glob('*.tif'))
        
        self.image_paths_list = image_paths_list
        self.number_of_images = len(self.image_paths_list)
        
        if self.number_of_images != self.number_of_pages:
            print(f'# of images != # of pages described in metadata')
            print(f'images: {self.number_of_images}')
            print(f'pages: {self.number_of_pages}')
            return None
        
        return self.image_paths_list
    
    def process(self):
        
        # progress bar
        progress_label = Label('Images being processed')
        progress_bar = IntProgress(min=0, max=self.number_of_images)
        progress_widget = VBox([progress_label, progress_bar])
        display(progress_widget)
        
        # get output directory name
        output_directory_name = df_tn_farm_and_home_science[df_tn_farm_and_home_science["old_directory_name"].str.contains(directory_path.name)]['new_directory_name'].to_string(index=False)
        # strip spaces from output_directory_name
        self.output_directory_name = output_directory_name.strip()
        # set output directory path and make directory
        self.output_directory_path = batch_output_directory_path.joinpath(self.output_directory_name)
        self.output_directory_path.mkdir()
        
        # for each *.tif in self.directory_path
        for index, image_path in enumerate(self.image_paths_list, start=1):
            
            # update progress bar label
            progress_label.value = f'Processing image: {image_path.name} . . . {index}/{self.number_of_images}'
            
            # NO extension at the end for Tesseract processing
            pdf_output_path = self.output_directory_path.joinpath(f'{str(index).zfill(3)}')
            
            # OCR with Tesseract
            !tesseract {str(image_path)} {str(pdf_output_path)} pdf 2>/dev/null
            
            # set output path with extension at end
            pdf_output_path = self.output_directory_path.joinpath(f'{str(index).zfill(3)}.pdf')
            
            # update progress bar value
            progress_bar.value = index
            
        self.pdf_paths_list = list(self.output_directory_path.glob('*.pdf'))
        self.number_of_pdfs = len(self.pdf_paths_list)
        
        if self.number_of_pdfs != self.number_of_pdfs:
            print(f'# of PDFs != # of images to OCR')
            print(f'PDFs: {self.number_of_pdfs}')
            print(f'images: {self.number_of_images}')
            return None
        else:
            return self.pdf_paths_list

In [17]:
# Set batch_input and batch_output directories

# set project identifier and root directory path
project_identifier = 'agrtfhs'
root_directory_path = Path('/Volumes/fluffy/ProjectCeres/00_for_CRL/')

# set batch_input and batch_output directory paths from root and project identifier
batch_input_directory_path = root_directory_path.joinpath(project_identifier)
batch_output_directory_path = root_directory_path.joinpath(f'{project_identifier}_batch_output')
batch_output_directory_path.mkdir()

# create batch_input directory paths list
batch_input_directory_paths_list = [x for x in batch_input_directory_path.iterdir() if x.is_dir()]
number_of_input_dirs = len(batch_input_directory_paths_list)

print(f'batch_input directory: {batch_input_directory_path}')
print(f'\t{number_of_input_dirs} directories to batch process\n')
print(f'batch_output directory: {batch_output_directory_path}')

batch_input directory: /Volumes/fluffy/ProjectCeres/00_for_CRL/agrtfhs
	135 directories to batch process

batch_output directory: /Volumes/fluffy/ProjectCeres/00_for_CRL/agrtfhs_batch_output


In [18]:
# start batch process

# progress bar
progress_label = Label('Directories being processed')
progress_bar = IntProgress(min=0, max=number_of_input_dirs)
progress_widget = VBox([progress_label, progress_bar])
display(progress_widget)

for index, directory_path in enumerate(batch_input_directory_paths_list, start=1):
    
    # update progress bar label
    progress_label.value = f'Processing directory: {directory_path.name} . . . {index}/{number_of_input_dirs}'
    
    # load directory path as class & process
    issue = Agrtfn(directory_path)
    issue.get_image_paths_list()
    issue.process()
    
    #update progress bar value
    progress_bar.value = index

print(f'Processed {index} directories')

Processed 135 directories


In [18]:
df

NameError: name 'df' is not defined

In [21]:
output_df = df_tn_farm_and_home_science['new_directory_name']

In [23]:
output_df.to_clipboard(index=False, header=False, excel=True, sep='\t')

In [24]:
from pathlib import Path